In [ ]:
import cv2
import opencv_jupyter_ui as jcv2
from feat import Detector
from IPython.display import Image
import joblib
from PIL import Image

from feat.utils import FEAT_EMOTION_COLUMNS

model_path ="../model/"
# Load the pre-trained RandomForest model
 
model = joblib.load(model_pkl)

# Initialize the pyfeat detector
detector = Detector(device="cuda")

# Initialize the webcam
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_BUFFERSIZE, 1)


def preprocess_au(au_values):
    # Ensure the AU values are in the correct format for the model
    # For example, if the model expects a specific number of AUs, you might need to adjust the input
    return np.array(au_values).reshape(1, -1)


while True:
    ret, frame = cam.read()
    if not ret:
        print("OpenCV found an error reading the next frame.")
        break

        # Detect faces, landmarks, and AUs using pyfeat
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Detect faces, landmarks, and AUs using pyfeat
    results = detector.detect_image([pil_image])
    print(results)

    # Display the frame
    jcv2.imshow("Emotion Detection", frame)

    # Wait for a key press
    key = jcv2.waitKey(1) & 0xFF
    if key == 27:  # ESC pressed
        break

# Release the webcam and close the window
cam.release()
jcv2.destroyAllWindows()

In [18]:
import cv2
import joblib
from feat import Detector
from PIL import Image
import numpy as np
import time

model_pkl = model_path + "model.pkl"
# Initialize the detector
detector = Detector()

# Emotion label mapping based on your training data
emotion_labels = {
    0: "neutral",
    1: "happy",
    2: "sad",
    3: "surprise",
    4: "fear",
    5: "disgust",
    6: "angry",
}


def extract_features(frame, detector):
    # Convert frame to format expected by py-feat (PIL Image)
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Detect faces and extract features
    detections = detector.detect_faces(frame_pil)
    if len(detections) == 0:
        return None

    detected_landmarks = detector.detect_landmarks(frame, detections)

    # Assuming that the model was trained using the features from the first detected face
    if len(detected_landmarks) > 0:
        aus = detector.detect_aus(frame_pil, detected_landmarks)
    else:
        return None

    # Check if AUs are extracted and handle the structure
    if isinstance(aus, list) and len(aus) > 0:
        # Flatten the structure if it's a nested list or array
        # and ensure only the expected number of features are returned
        aus_flat = np.array(aus[0]).flatten()[
            :171
        ]  # Adjust number of features if needed
        return aus_flat
    else:
        return None


# Load the trained model
model = joblib.load(model_pkl)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Set desired FPS
desired_fps = 10  # Adjust as needed
frame_interval = 1 / desired_fps

last_time = time.time()

while True:

    # Control frame rate
    current_time = time.time()
    if current_time - last_time < frame_interval:
        continue
    last_time = current_time

    ret, frame = cap.read()
    if not ret:
        break
    # Preprocess the frame and extract features
    features = extract_features(frame, detector)
    print("Features:", features)
    # Check if features are extracted
    if features is not None and len(features) > 0:
        # Predict emotion
        emotion = model.predict([features])[0]

        emotion = emotion_labels.get(emotion, "Unknown")

        # Display the result
        cv2.putText(
            frame,
            f"Emotion: {emotion}",
            (10, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2,
        )

    # Show the frame
    cv2.imshow("Emotion Recognition", frame)

    # Break the loop with 'q' key
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

INFO:root:Loading Face model: retinaface
c:\Users\amrit\env\Lib\site-packages\feat\face_detectors\Retinaface\Retinaface_test.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

Features: [5.4484105e-01 3.0810755e-01 1.7176951e-01 2.8228968e-01 1.4600801e-01
 0.0000000e+00 2.5567955e-01 1.2390041e-01 0.0000000e+00 6.1925329e-02
 5.9680843e-01 8.1709290e-01 7.1526265e-01 0.0000000e+00 7.8586507e-01
 9.2497569e-01 4.9439963e-04 7.9481013e-02 9.1741371e-01 1.9230428e-01]


ValueError: X has 20 features, but SimpleImputer is expecting 171 features as input.